# Train A Shape Classifier Model



In [ ]:
import json
import os

train_data_root = "../datasets/train"
test_data_root = "../datasets/test"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations (including resizing and normalization)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale (black and white images)
    transforms.Resize((64, 64)),  # Resize images to 64x64 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images (mean=0.5, std=0.5 for grayscale)
])

# Load the dataset
train_dataset = datasets.ImageFolder(root=train_data_root, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_root, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class names (optional)
print(f'Classes: {train_dataset.classes}')

# 2. Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) #bug 1
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 3)  # 3 classes: circle, triangle, rectangle
        
    def forward(self, x):
        x = F.relu(self.conv1(x))   # First Conv Layer
        x = F.max_pool2d(x, 2)      # Max Pooling
        x = F.relu(self.conv2(x))   # Second Conv Layer
        x = F.max_pool2d(x, 2)      # Max Pooling
        x = x.view(x.size(0), -1)   # Flatten
        x = F.relu(self.fc1(x))     # Fully Connected Layer 1
        x = self.fc2(x)             # Fully Connected Layer 2 (output)
        return x

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total #bug 2
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')


# Train

In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=15)

# Test 

In [ ]:
def test(model, test_loader):
    """Print the Precision, Recall and F1-score for the trained model
    """
    
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score

    # Set the model to evaluation mode
    model.eval()

    all_predictions = []
    all_targets = []

    # Evaluate on test data
    with torch.no_grad():
        for batch_images, batch_labels in test_loader:
            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
            outputs = model(batch_images)
            _, preds = torch.max(outputs, 1)

            all_predictions.extend(preds.cpu().numpy())
            all_targets.extend(batch_labels.cpu().numpy())

    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average=None)
    acc = accuracy_score(all_targets, all_predictions)

    # Print results per class
    print("Result:")
    for idx, cls_name in enumerate(test_loader.dataset.classes):
        print(f"{cls_name}:")
        print(f"  Precision: {precision[idx]:.4f}")
        print(f"  Recall:    {recall[idx]:.4f}")
        print(f"  F1-score:  {f1[idx]:.4f}")

    print(f"\nOverall Accuracy: {acc:.4f}")
    
    pass

test(model, test_loader)

# Show Predictions


In [ ]:
def show_prediction(model, image):
    """Pass the image to the model and overlay the predicted shape and confidence on the input
    image and display it
    """
    import matplotlib.pyplot as plt
    import numpy as np

    # Convert numpy array to tensor if needed
    if isinstance(image, np.ndarray):
        image = torch.tensor(image, dtype=torch.float32)

    # Ensure batch and channel dimensions
    if len(image.shape) == 2:        # H x W
        image = image.unsqueeze(0).unsqueeze(0)
    elif len(image.shape) == 3:      # C x H x W
        image = image.unsqueeze(0)

    # Normalize the image like during training
    image = (image / 255.0 - 0.5) / 0.5
    image = image.to(device)

    # Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        probs = F.softmax(outputs, dim=1)
        confidence, pred = torch.max(probs, 1)

    class_name = test_loader.dataset.classes[pred.item()]

    # Display image with prediction overlay
    plt.figure(figsize=(6,6))
    img_display = image[0,0].cpu().numpy()
    plt.imshow(img_display, cmap='gray')
    plt.axis('off')
    plt.text(
        5, 10,
        f"{class_name} ({confidence.item():.2%})",
        color='red',
        fontsize=14,
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='red', boxstyle='round,pad=0.3')
    )
    plt.show()
    
    pass

test_images, test_labels = next(iter(test_loader))
show_prediction(model, test_images[0])